In [11]:
import pandas as pd
import numpy as np
import math 
import matplotlib 
from sklearn.metrics.pairwise import cosine_similarity

### a) Write a Python (or Java) program to implement a user-based Collaborative Filtering Recommender System that takes a movie name as input and recommends top-5 movies to the current user. Use Cosine Similarity for finding neighborhoods and mean-centered prediction function (refer to Lecture 2 slides for context).

## Loading Dataset

In [4]:
links = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\A1\ml-latest-small\links.csv")
ratings = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\A1\ml-latest-small\ratings.csv")
movies = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\A1\ml-latest-small\movies.csv")
tags = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\A1\ml-latest-small\tags.csv")

In [6]:
links.head(6)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
5,6,113277,949.0


In [10]:
tags.head(6)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205


In [7]:
ratings.head(6)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400


In [8]:
movies.head(6)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller


## Preprocessing

In [76]:
#after looking at all datasets in the ml_latest files, for our tasks now, we will use ratings and movies dataset.

In [189]:
mergedData = pd.merge(ratings, movies, on='movieId', how='inner')
finalData = mergedData.pivot_table(index='userId', columns='title', values='rating')
finalData.tail(5)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN
609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,...,NaN,4.0,3.5,3.0,NaN,NaN,2.0,1.5,NaN,NaN


In [75]:
#Normalizing a big dataaset was taking so much time and kernel became unresponsive
# scaled_finalData = finalData.copy()

# for column in scaled_finalData.columns:
#     scaled_finalData[column] = (scaled_finalData[column] - scaled_finalData[column].mean()) / scaled_finalData[column].std()    
     
# print(scaled_finalData)

In [74]:
print(mergedData['title'].nunique()) #these are a lot of movies to utilise for computation

9719


In [88]:
df = mergedData.groupby('title').agg(avg_rating=('rating','mean'),total_ratings=('rating','count')).reset_index()
df = df[df['total_ratings']>150]
newdf = pd.merge(df, df[['title']], on='title', how='inner')

In [93]:
print(newdf['title'].nunique()) #reduced from 9719 to 43 movies to work with.

43


In [177]:
new_merge = pd.merge(mergedData, newdf[['title']], on='title', how='inner')

In [95]:
finalData = new_merge.pivot_table(index='userId', columns='title', values='rating')
finalData.tail(10)

title,Ace Ventura: Pet Detective (1994),Aladdin (1992),American Beauty (1999),Apollo 13 (1995),Back to the Future (1985),Batman (1989),Braveheart (1995),Dances with Wolves (1990),Fargo (1996),Fight Club (1999),...,"Sixth Sense, The (1999)",Speed (1994),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Terminator 2: Judgment Day (1991),Toy Story (1995),True Lies (1994),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,4.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0
602,2.0,NaN,NaN,4.0,NaN,4.0,5.0,3.0,5.0,NaN,...,NaN,4.0,5.0,NaN,NaN,5.0,NaN,3.0,3.0,5.0
603,NaN,NaN,5.0,NaN,2.0,2.0,1.0,3.0,5.0,4.0,...,5.0,5.0,4.0,3.0,3.0,NaN,4.0,2.0,3.0,NaN
604,2.0,3.0,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,4.0,NaN
605,NaN,3.5,NaN,5.0,4.0,3.0,3.0,3.0,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,3.5,4.0,3.0,NaN,NaN
606,NaN,NaN,4.5,NaN,3.5,3.5,3.5,4.0,NaN,5.0,...,4.0,NaN,4.5,4.5,4.5,3.5,2.5,NaN,4.0,4.5
607,NaN,NaN,3.0,5.0,3.0,3.0,5.0,NaN,2.0,NaN,...,5.0,5.0,3.0,3.0,3.0,4.0,4.0,4.0,NaN,NaN
608,3.5,3.0,5.0,2.0,2.0,3.0,4.0,3.0,3.0,5.0,...,4.5,2.5,3.5,4.0,4.0,3.0,2.5,3.0,3.5,4.5
609,NaN,NaN,NaN,3.0,NaN,3.0,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN


In [129]:
scaled_finalData = finalData.copy()

for column in scaled_finalData.columns:
    scaled_finalData[column] = (scaled_finalData[column] - scaled_finalData[column].mean()) / scaled_finalData[column].std()    
     
scaled_finalData

title,Ace Ventura: Pet Detective (1994),Aladdin (1992),American Beauty (1999),Apollo 13 (1995),Back to the Future (1985),Batman (1989),Braveheart (1995),Dances with Wolves (1990),Fargo (1996),Fight Club (1999),...,"Sixth Sense, The (1999)",Speed (1994),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Terminator 2: Judgment Day (1991),Toy Story (1995),True Lies (1994),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Usual Suspects, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.972560,NaN,1.100063,0.737758,-0.032701,0.191435,1.066922,0.844065,...,NaN,NaN,0.881790,0.931369,0.995530,NaN,0.094710,NaN,NaN,0.951723
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.235825,0.972560,NaN,NaN,NaN,NaN,NaN,1.066922,-2.638701,...,0.111098,NaN,0.881790,0.931369,NaN,NaN,NaN,NaN,-2.508963,NaN
5,-0.037700,0.235825,NaN,-0.964727,NaN,-0.553318,-0.032701,1.354225,-1.346988,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.047634,0.094710,-1.64149,NaN,-0.296840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,0.457229,NaN,-0.615233,0.092220,-0.549376,0.191435,NaN,0.844065,...,0.111098,NaN,0.308398,0.337657,0.418241,-0.508163,-1.702000,NaN,0.021444,0.327442
607,NaN,NaN,-1.088762,1.316568,-1.186998,-0.553318,1.000649,NaN,-2.553944,NaN,...,1.157758,1.574407,-1.411778,-1.443482,-1.313629,0.031309,0.094710,0.55127,NaN,NaN
608,0.429203,-0.899860,0.972560,-2.105374,-2.330529,-0.553318,-0.032701,-0.971355,-1.346988,0.844065,...,0.634428,-1.101772,-0.838386,-0.256056,-0.159049,-1.047634,-1.702000,-0.54511,-0.611158,0.327442


## Similarity

In [130]:
similarity = pd.DataFrame(cosine_similarity(scaled_finalData.fillna(0)), 
                          index = scaled_finalData.index,
                          columns = scaled_finalData.index )
similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.007853,-0.188512,0.039061,-0.095425,-0.059249,-0.099335,0.020657,0.251800,-0.156254,...,0.231805,0.343786,-0.263798,-0.084733,0.260124,0.032771,-0.221977,-0.255929,-0.037542,0.363784
2,0.007853,1.000000,0.000000,0.000000,0.498968,-0.120451,-0.017168,-0.204289,0.000000,0.000232,...,-0.244518,-0.173227,0.000280,0.000000,0.000000,0.306065,-0.145824,-0.017057,0.246819,0.337561
3,-0.188512,0.000000,1.000000,0.000000,-0.197787,0.188881,0.000000,-0.202669,0.000000,0.000000,...,-0.248002,-0.171854,0.182744,0.000000,0.000000,-0.188594,-0.144668,0.040299,0.000000,0.123140
4,0.039061,0.000000,0.000000,1.000000,-0.141492,0.162480,0.030540,0.067440,-0.098587,0.519819,...,-0.287270,0.062285,-0.054428,-0.111777,0.264658,-0.038445,-0.045331,-0.279643,0.039292,-0.284891
5,-0.095425,0.498968,-0.197787,-0.141492,1.000000,0.006565,0.106965,-0.236156,0.000000,-0.083544,...,-0.097082,-0.147264,0.028139,0.069926,-0.125276,0.254583,-0.023023,0.186812,0.363451,-0.027273


In [185]:
active_user = int(input())
print("active user is user:",active_user)
k = int(input())
print("K neighbors:",k)
Q = float(input())
print("threshold:",Q)

1
active user is user: 1
5
K neighbors: 5
0.4
threshold: 0.4


In [186]:
similar_users = similarity[similarity[active_user] > Q][active_user].sort_values(ascending=False)[:k]
print('Similar users for user', active_user,' are', similar_users)

Similar users for user 1  are userId
414    0.490181
95     0.485074
13     0.478028
400    0.464978
597    0.457675
Name: 1, dtype: float64


In [187]:
user_watched = scaled_finalData[scaled_finalData.index == active_user].dropna(axis=1, how='all')
similar_user_movies = scaled_finalData[scaled_finalData.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_movies.drop(user_watched.columns,axis=1, inplace=True, errors='ignore')

In [188]:
# A dictionary to store item scores
item_score = {}
# Loop through items
for i in similar_user_movies.columns:
  # Get the ratings for movie i
  movie_rating = similar_user_movies[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in similar_users.index:
    # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
      # Score is the sum of user similarity score multiply by the movie rating
      score = similar_users[u] * movie_rating[u]
      # Add the score to the total score for the movie so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the item
  item_score[i] = total / count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
# Select top m movies
m = 5
ranked_item_score.head(m)

,movie,movie_score
8,Memento (2000),0.475297
7,"Lord of the Rings: The Two Towers, The (2002)",0.398917
9,"Shawshank Redemption, The (1994)",0.382441
3,"Godfather, The (1972)",0.370227
13,Terminator 2: Judgment Day (1991),0.355849


### b) Write a Python (or Java) program to implement a Content-based Recommender System that takes a movie name as input and recommends top-3 movies to the user. Use genres as features and the provided ratings for vectorization of all profiles (refer to Lecture 3 slides for context).